## Creando tablas de tipos compuestos

In [1]:
! mkdir -p hive-tablastiposcompuestos

In [2]:
import os
os.chdir("hive-tablastiposcompuestos")

In [3]:
! pwd

/media/notebooks/hive-tablastiposcompuestos


In [5]:
! hadoop fs -put ../empleados.txt datoshive

In [18]:
! hadoop fs -ls /user/$(whoami)/datoshive/empleados.txt

-rw-r--r--   1 root supergroup        264 2019-04-17 11:21 /user/root/datoshive/empleados.txt


In [11]:
%%writefile ejerciciohive.hql
create database if not exists bdempleados
Comment 'BD de empleados'
Location '/user/$(whoami)/empleados'
With dbproperties ('Creada por'='User','Creada el'='26-Dic-2017');


Overwriting ejerciciohive.hql


In [12]:
! beeline -u "jdbc:hive2://localhost:10000/default" -f ejerciciohive.hql

2019-04-17 11:24:08,093 WARN  [main] mapreduce.TableMapReduceUtil: The hbase-prefix-tree module jar containing PrefixTreeCodec is not present.  Continuing without it.
scan complete in 3ms
Connecting to jdbc:hive2://localhost:10000/default
Connected to: Apache Hive (version 1.1.0-cdh5.7.0)
Driver: Hive JDBC (version 1.1.0-cdh5.7.0)
Transaction isolation: TRANSACTION_REPEATABLE_READ
os jdbc:hive2://localhost:10000/default> create database if not exists bdemplead 
0: jdbc:hive2://localhost:10000/default> Comment 'BD de empleados'
0: jdbc:hive2://localhost:10000/default> Location '/user/$(whoami)/empleados'
'Creada el'='26-Dic-2017');0000/default> With dbproperties ('Creada por'='User', 
INFO  : Compiling command(queryId=hive_20190417112424_4b35fc8f-8097-4a03-9287-a7663fd9edb6): create database if not exists bdempleados
Comment 'BD de empleados'
Location '/user/$(whoami)/empleados'
With dbproperties ('Creada por'='User','Creada el'='26-Dic-2017')
INFO  : Semantic Analysis Completed
INFO  :

In [13]:
%%writefile ejerciciohive.hql
CREATE TABLE IF NOT EXISTS empleadosinterna
(
  nombre string,
  ciudad ARRAY<string>,
  sexo_edad STRUCT<sexo:string,edad:int>,
  habilidades_puntuacion MAP<string,int>,
  dpto_rol MAP<STRING,STRING>
)
COMMENT 'Tabla interna'
ROW FORMAT DELIMITED
FIELDS TERMINATED BY '|'
COLLECTION ITEMS TERMINATED BY ','
MAP KEYS TERMINATED BY ':';

LOAD DATA INPATH '/user/root/datoshive/empleados.txt' INTO TABLE empleadosinterna;


show tables;

Overwriting ejerciciohive.hql


In [14]:
! beeline -u "jdbc:hive2://localhost:10000/bdempleados" -f ejerciciohive.hql

2019-04-17 11:24:37,453 WARN  [main] mapreduce.TableMapReduceUtil: The hbase-prefix-tree module jar containing PrefixTreeCodec is not present.  Continuing without it.
scan complete in 2ms
Connecting to jdbc:hive2://localhost:10000/bdempleados
Connected to: Apache Hive (version 1.1.0-cdh5.7.0)
Driver: Hive JDBC (version 1.1.0-cdh5.7.0)
Transaction isolation: TRANSACTION_REPEATABLE_READ
sinternahive2://localhost:10000/bdempleados> CREATE TABLE IF NOT EXISTS empleado 
0: jdbc:hive2://localhost:10000/bdempleados> (
0: jdbc:hive2://localhost:10000/bdempleados>   nombre string,
0: jdbc:hive2://localhost:10000/bdempleados>   ciudad ARRAY<string>,
:int>,c:hive2://localhost:10000/bdempleados>   sexo_edad STRUCT<sexo:string,edad 
,int>,c:hive2://localhost:10000/bdempleados>   habilidades_puntuacion MAP<string 
0: jdbc:hive2://localhost:10000/bdempleados>   dpto_rol MAP<STRING,STRING>
0: jdbc:hive2://localhost:10000/bdempleados> )
0: jdbc:hive2://localhost:10000/bdempleados> COMMENT 'Tabla intern

In [20]:
! beeline -u "jdbc:hive2://localhost:10000/bdempleados" -e "select * from empleadosinterna;"

2019-04-17 11:28:07,575 WARN  [main] mapreduce.TableMapReduceUtil: The hbase-prefix-tree module jar containing PrefixTreeCodec is not present.  Continuing without it.
scan complete in 2ms
Connecting to jdbc:hive2://localhost:10000/bdempleados
Connected to: Apache Hive (version 1.1.0-cdh5.7.0)
Driver: Hive JDBC (version 1.1.0-cdh5.7.0)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=hive_20190417112828_78a12b13-0b6e-4859-9625-47b2a463f7e3): select * from empleadosinterna
INFO  : Semantic Analysis Completed
INFO  : Returning Hive schema: Schema(fieldSchemas:[FieldSchema(name:empleadosinterna.nombre, type:string, comment:null), FieldSchema(name:empleadosinterna.ciudad, type:array<string>, comment:null), FieldSchema(name:empleadosinterna.sexo_edad, type:struct<sexo:string,edad:int>, comment:null), FieldSchema(name:empleadosinterna.habilidades_puntuacion, type:map<string,int>, comment:null), FieldSchema(name:empleadosinterna.dpto_rol, type:map<string,str

Devuelve el nombre de los empleados de Madrid

In [22]:
! beeline -u "jdbc:hive2://localhost:10000/bdempleados" -e "select nombre from empleadosinterna e where array_contains(e.ciudad, 'Madrid');"

2019-04-17 11:33:29,652 WARN  [main] mapreduce.TableMapReduceUtil: The hbase-prefix-tree module jar containing PrefixTreeCodec is not present.  Continuing without it.
scan complete in 3ms
Connecting to jdbc:hive2://localhost:10000/bdempleados
Connected to: Apache Hive (version 1.1.0-cdh5.7.0)
Driver: Hive JDBC (version 1.1.0-cdh5.7.0)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=hive_20190417113333_61806197-8bad-4af0-865e-140dc27a79b1): select nombre from empleadosinterna e where array_contains(e.ciudad, 'Madrid')
INFO  : Semantic Analysis Completed
INFO  : Returning Hive schema: Schema(fieldSchemas:[FieldSchema(name:nombre, type:string, comment:null)], properties:null)
INFO  : Completed compiling command(queryId=hive_20190417113333_61806197-8bad-4af0-865e-140dc27a79b1); Time taken: 0.069 seconds
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Executing command(queryId=hive_20190417113333_61806197-8bad-4af0-865e-140dc27a

Devuelve el nombre de los empleados feneminos. 

In [26]:
! beeline -u "jdbc:hive2://localhost:10000/bdempleados" -e "select nombre from empleadosinterna where sexo_edad.sexo='Femenino';"

2019-04-17 11:38:18,911 WARN  [main] mapreduce.TableMapReduceUtil: The hbase-prefix-tree module jar containing PrefixTreeCodec is not present.  Continuing without it.
scan complete in 3ms
Connecting to jdbc:hive2://localhost:10000/bdempleados
Connected to: Apache Hive (version 1.1.0-cdh5.7.0)
Driver: Hive JDBC (version 1.1.0-cdh5.7.0)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=hive_20190417113838_ad3811be-97a0-4672-ae0e-6de7f77f16bb): select nombre from empleadosinterna where sexo_edad.sexo='Femenino'
INFO  : Semantic Analysis Completed
INFO  : Returning Hive schema: Schema(fieldSchemas:[FieldSchema(name:nombre, type:string, comment:null)], properties:null)
INFO  : Completed compiling command(queryId=hive_20190417113838_ad3811be-97a0-4672-ae0e-6de7f77f16bb); Time taken: 0.113 seconds
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Executing command(queryId=hive_20190417113838_ad3811be-97a0-4672-ae0e-6de7f77f16bb): sele

Devuelve el nombre del jefe del departamento de Produccion.

In [28]:
! beeline -u "jdbc:hive2://localhost:10000/bdempleados" -e "select nombre from empleadosinterna where dpto_rol['Produccion']='Jefe';"

2019-04-17 11:40:38,933 WARN  [main] mapreduce.TableMapReduceUtil: The hbase-prefix-tree module jar containing PrefixTreeCodec is not present.  Continuing without it.
scan complete in 2ms
Connecting to jdbc:hive2://localhost:10000/bdempleados
Connected to: Apache Hive (version 1.1.0-cdh5.7.0)
Driver: Hive JDBC (version 1.1.0-cdh5.7.0)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=hive_20190417114040_0909ba40-a8e4-4bcd-8759-f10da54cdf2a): select nombre from empleadosinterna where dpto_rol['Produccion']='Jefe'
INFO  : Semantic Analysis Completed
INFO  : Returning Hive schema: Schema(fieldSchemas:[FieldSchema(name:nombre, type:string, comment:null)], properties:null)
INFO  : Completed compiling command(queryId=hive_20190417114040_0909ba40-a8e4-4bcd-8759-f10da54cdf2a); Time taken: 0.087 seconds
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Executing command(queryId=hive_20190417114040_0909ba40-a8e4-4bcd-8759-f10da54cdf2a): 

## Implementa los siguientes ejercicios

Devuelve los nombres de los empleados del departamento de Produccion

Devuelve el nombre de los empleados que tengan entre 70 y 90 de habilidad con Python